In [2]:
import pgmpy

Monty Hall problem就是那个3扇门、2只山羊和一辆车那个问题。

现在我们有3个RVs，其中第一个C表示选手会选择的门编号，第二个H表示支持人会选择的门的编号，第三个是有奖励的门的编号。根据该问题的描述，我们可以得到下面的贝叶斯网络图

![](./monty.png)

和下面的CPDs：

![](./CPDs.png)

我们来计算一下选手选择了门0，主持人打开了门2，现在我们看选手找到奖励的概率

In [7]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

model = BayesianModel([("C", "H"), ("P", "H")])

cpd_c = TabularCPD("C", 3, [[0.33, 0.33, 0.33]])
cpd_p = TabularCPD("P", 3, [[0.33, .33, .33]])
cpd_h = TabularCPD('H', 3, [[0, 0, 0, 0, 0.5, 1, 0, 1, 0.5], 
                            [0.5, 0, 1, 0, 0, 0, 1, 0, 0.5], 
                            [0.5, 1, 0, 1, 0.5, 0, 0, 0, 0]],
                  evidence=['C', 'P'], evidence_card=[3, 3])
model.add_cpds(cpd_c, cpd_p, cpd_h)
# 检查model的结构和CPD是否是正确（加起来是1）
model.check_model()

True

In [13]:
from pgmpy.inference import VariableElimination

infer = VariableElimination(model)
posterior_p = infer.query(["P"], evidence={"C": 1, "H": 2})
print(posterior_p)

Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]


+------+----------+
| P    |   phi(P) |
+======+==========+
| P(0) |   0.6667 |
+------+----------+
| P(1) |   0.3333 |
+------+----------+
| P(2) |   0.0000 |
+------+----------+


发现，我们换门会增加获奖的概率。我们也可以将选择的门的编号改为1，会发现相同的结论。